In [2]:
#Importing The Dataset

In [3]:
import pandas as pd
import sys

In [4]:
Pledge = pd.read_csv('C:/Users/soubh/Downloads/OOPS/Plotly x OFTW/CSV/Pledge.csv')

In [5]:
Payments = pd.read_csv('C:/Users/soubh/Downloads/OOPS/Plotly x OFTW/CSV/Payments.csv')

In [6]:
#Removing Discripency From The Data

In [7]:
Pledge = Pledge.drop(Pledge.index[0], axis=0)

In [8]:
#Converting Date Into Pandas Date

In [9]:
Payments['date'] = pd.to_datetime(Payments['date'])

In [10]:
Pledge['pledge_created_at'] = pd.to_datetime(Pledge['pledge_created_at'])
Pledge['pledge_starts_at'] = pd.to_datetime(Pledge['pledge_starts_at'])
Pledge['pledge_ended_at'] = pd.to_datetime(Pledge['pledge_ended_at'])

In [11]:
#Converting The Currency Of The Datasets Pledge Completely To USD

In [12]:
def load_exchange_rates():
    gbp = pd.read_csv('ExchangeRate_CSV/DEXUSUK_exchange_rates.csv')
    cad = pd.read_csv('ExchangeRate_CSV/DEXCAUS_exchange_rates.csv')
    aud = pd.read_csv('ExchangeRate_CSV/DEXUSAL_exchange_rates.csv')
    eur = pd.read_csv('ExchangeRate_CSV/DEXUSEU_exchange_rates.csv')
    sgd = pd.read_csv('ExchangeRate_CSV/DEXSIUS_exchange_rates.csv')
    chf = pd.read_csv('ExchangeRate_CSV/DEXSZUS_exchange_rates.csv')
    
    for df in [gbp, cad, aud, eur, sgd, chf]:
        df.rename(columns={'DATE': 'date'}, inplace=True)
        df['date'] = pd.to_datetime(df['date'])
    
    return gbp, cad, aud, eur, sgd, chf

In [13]:
def convert_payments_to_usd(payments_df, gbp, cad, aud, eur, sgd, chf):
    df = payments_df.copy()
    
    # Merge exchange rates
    df = df.merge(gbp, on='date', how='left')
    df = df.merge(cad, on='date', how='left')
    df = df.merge(aud, on='date', how='left')
    df = df.merge(eur, on='date', how='left')
    df = df.merge(sgd, on='date', how='left')
    df = df.merge(chf, on='date', how='left')

    # Convert to USD
    df['amount_usd'] = df.apply(
        lambda row: row['amount'] * row['DEXUSUK'] if row['currency'] == 'GBP' else
                    row['amount'] / row['DEXCAUS'] if row['currency'] == 'CAD' else
                    row['amount'] * row['DEXUSAL'] if row['currency'] == 'AUD' else
                    row['amount'] * row['DEXUSEU'] if row['currency'] == 'EUR' else
                    row['amount'] / row['DEXSIUS'] if row['currency'] == 'SGD' else
                    row['amount'] / row['DEXSZUS'] if row['currency'] == 'CHF' else
                    row['amount'], axis=1
    )

    # Drop exchange rate columns
    df.drop(columns=['DEXUSUK', 'DEXCAUS', 'DEXUSAL', 'DEXUSEU', 'DEXSIUS', 'DEXSZUS'], inplace=True)
    return df

In [14]:
#Converting The Amount To USD From The Pledge Dataset

In [15]:
def convert_pledges_to_usd(pledge_df, gbp, cad, aud, eur, sgd, chf):
    df = pledge_df.copy()
    df['pledge_created_at'] = pd.to_datetime(df['pledge_created_at'])
    df.rename(columns={'pledge_created_at': 'date'}, inplace=True)
    
    # Merge exchange rates
    df = df.merge(gbp, on='date', how='left')
    df = df.merge(cad, on='date', how='left')
    df = df.merge(aud, on='date', how='left')
    df = df.merge(eur, on='date', how='left')
    df = df.merge(sgd, on='date', how='left')
    df = df.merge(chf, on='date', how='left')

    # Convert to USD
    df['amount_usd'] = df.apply(
        lambda row: row['contribution_amount'] * row['DEXUSUK'] if row['currency'] == 'GBP' else
                    row['contribution_amount'] / row['DEXCAUS'] if row['currency'] == 'CAD' else
                    row['contribution_amount'] * row['DEXUSAL'] if row['currency'] == 'AUD' else
                    row['contribution_amount'] * row['DEXUSEU'] if row['currency'] == 'EUR' else
                    row['contribution_amount'] / row['DEXSIUS'] if row['currency'] == 'SGD' else
                    row['contribution_amount'] / row['DEXSZUS'] if row['currency'] == 'CHF' else
                    row['contribution_amount'], axis=1
    )

    # Drop exchange rate columns
    df.drop(columns=['DEXUSUK', 'DEXCAUS', 'DEXUSAL', 'DEXUSEU', 'DEXSIUS', 'DEXSZUS'], inplace=True)
    return df

In [16]:
# Load exchange rates
gbp_df, cad_df, aud_df, eur_df, sgd_df, chf_df = load_exchange_rates()

# Convert
df_payments_converted = convert_payments_to_usd(Payments, gbp_df, cad_df, aud_df, eur_df, sgd_df, chf_df)
df_pledges_converted = convert_pledges_to_usd(Pledge, gbp_df, cad_df, aud_df, eur_df, sgd_df, chf_df)

In [17]:
#Segregating Payments Tables based on the Null Values of pledge_id and Non_Null values of pledge_id

In [18]:
#Payments With Pledge Null Values
Anonymous = df_payments_converted[df_payments_converted['pledge_id'].isna()]

In [19]:
Pledged_Payments = df_payments_converted[df_payments_converted['pledge_id'].notna()]

In [20]:
#Merging Both The Tables

In [21]:
Merged_INNER = pd.merge(df_pledges_converted, df_payments_converted, on='pledge_id', how='inner')

In [22]:
Merged_LEFT = pd.merge(df_pledges_converted, Pledged_Payments, on='pledge_id', how='left')

In [23]:
Merged_INNER['amount_usd_x'].sum()

np.float64(8435090.93604322)

In [24]:
Merged_INNER['amount_usd_y'].sum()

np.float64(8235604.838483917)

In [25]:
total_pledged = df_pledges_converted['amount_usd'].sum()
print(f"Total Pledged Amount (USD): {total_pledged}")

Total Pledged Amount (USD): 4748359.601655179


In [26]:
total_paid = df_payments_converted['amount_usd'].sum()
print(f"Total Paid Amount (USD): {total_paid}")

Total Paid Amount (USD): 8228034.6181357335


In [27]:
total_pledged_by_pledge = df_pledges_converted.groupby('pledge_id')['amount_usd'].sum()
total_paid_by_pledge = df_payments_converted.groupby('pledge_id')['amount_usd'].sum()

comparison = pd.DataFrame({
    'pledged': total_pledged_by_pledge,
    'paid': total_paid_by_pledge
}).fillna(0)

comparison['difference'] = comparison['pledged'] - comparison['paid']
print(comparison.head())  # View the comparison for a few pledges

                                      pledged     paid  difference
pledge_id                                                         
000d0331-481c-4131-9eaa-e0faa91700b4     20.0    120.0      -100.0
000f3ad6-ce57-42ba-adc2-9ccc0eae5a05      0.0      0.0         0.0
0014a95e-bddf-491b-81fc-c0c61bb67607    129.0    387.0      -258.0
0016baae-c4a9-4098-b2e1-2ee32d0069fd  10000.0  10000.0         0.0
0017c19b-080e-4eb8-80e1-2d20d350d7c2     54.0      0.0        54.0


In [28]:
df_pledges_converted['pledge_id'].nunique()

11824

In [29]:
df_payments_converted['pledge_id'].nunique()

6321

In [31]:
df_pledges_converted

,donor_id,pledge_id,donor_chapter,chapter_type,pledge_status,date,pledge_starts_at,pledge_ended_at,contribution_amount,currency,frequency,payment_platform,amount_usd
0,NaN,fa0e5a1b-ab1d-4aba-af20-2b9d6aa23312,NaN,NaN,One-Time,2016-12-24,2016-12-24,NaT,4000.0,USD,One-Time,Fidelity DAF,4000.000
1,NaN,71bfd12a-7877-4e21-a347-7646f823149f,NaN,NaN,One-Time,2016-12-24,2016-12-24,NaT,1250.0,USD,One-Time,Fidelity DAF,1250.000
2,NaN,f8811c05-5dfd-4760-b730-8532beb2f34d,NaN,NaN,One-Time,2016-12-24,2016-12-24,NaT,1250.0,USD,One-Time,Fidelity DAF,1250.000
3,NaN,58bf5aec-b09c-4a43-ba9d-6197c1bfd7ef,NaN,NaN,One-Time,2016-12-24,2016-12-24,NaT,4000.0,USD,One-Time,Fidelity DAF,4000.000
4,NaN,32d7ecbb-e909-4881-9dd1-0875fe78fa33,NaN,NaN,One-Time,2017-07-22,2017-07-22,NaT,3000.0,USD,One-Time,Fidelity DAF,3000.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11905,NaN,bb88d99c-e831-462c-b9c4-2209a819a368,NaN,NaN,One-Time,2024-12-04,2024-12-04,NaT,60.0,GBP,One-Time,Benevity,76.272
11906,NaN,761c7af3-1b01-4852-9da0-76bd570d0fee,ZS Associates,NaN,Updated,2022-08-18,2022-08-18,2022-08-24,138.0,USD,Monthly,Benevity,138.000
11907,NaN,619cc7d3-79a0-4b3c-a027-999f902d6a84,ZS Associates,NaN,Updated,2022-10-26,2022-10-26,2023-06-28,167.0,USD,Monthly,Benevity,167.000
11908,NaN,a972e21e-9d54-477e-9385-809628f56320,ZS Associates,NaN,Updated,2023-06-27,2023-06-27,2023-10-27,350.0,USD,Monthly,Benevity,350.000


In [38]:
# Ensure date column is datetime
df_payments_converted["date"] = pd.to_datetime(df_payments_converted["date"])

# Define fiscal year start and end for FY24–25
fy_start = pd.Timestamp("2023-07-01")
fy_end = pd.Timestamp("2024-06-30")

# Filter for FY24–25
fy_mask = (df_payments_converted["date"] >= fy_start) & (df_payments_converted["date"] <= fy_end)
df_fy = df_payments_converted.loc[fy_mask].copy()

# Monthly Money Moved
monthly_mm = df_fy.groupby(df_fy["date"].dt.to_period("M"))["amount_usd"].sum().reset_index()
monthly_mm["date"] = monthly_mm["date"].dt.to_timestamp()

# Total FYTD Money Moved
total_fytd = df_fy["amount_usd"].sum()

In [39]:
print(total_fytd)

1703280.726534693


In [41]:
df_pledges_converted['chapter_type'].isna().sum()

np.int64(2439)

In [42]:
df_payments_converted

,id,donor_id,payment_platform,portfolio,amount,currency,date,counterfactuality,pledge_id,amount_usd
0,6d99138e-3f1b-49db-944a-9d13c6733ea4,NaN,Squarespace,OFTW Top Picks,0.00,USD,2025-01-11,0.758170,NaN,0.00
1,264TJ405CZ,NaN,Benevity,OFTW Top Picks,0.00,USD,2020-09-27,0.313636,NaN,0.00
2,5B5XGD4RTX,NaN,Benevity,OFTW Top Picks,-100.00,USD,2024-02-01,0.313636,7373fca9-78cc-4954-baaa-b2acebac595b,-100.00
3,3Z5NNTTCSD,NaN,Benevity,OFTW Top Picks,-1.01,USD,2022-09-23,0.313636,5e6bc702-7990-4351-a5b5-4cf39401c379,-1.01
4,31V9VD2AAV,NaN,Benevity,OFTW Top Picks,0.00,USD,2021-11-19,0.313636,8a8c1bc3-a6a4-4819-a8c1-732e7a88408b,0.00
...,...,...,...,...,...,...,...,...,...,...
55342,77979a46-6783-4278-899f-f3cd713bbefc,NaN,Off Platform,OFTW Top Pick: Against Malaria Foundation,127496.00,USD,2021-10-15,0.758170,c252140c-c515-454e-b9be-21385a4561b2,127496.00
55343,2f01a32e-9252-4640-a3ae-21cfd59ff67c,NaN,Off Platform,GiveWell (Maximum Impact Fund),135000.00,USD,2022-12-16,0.758170,10bec868-da3d-468e-a765-41e22f2b5a9d,135000.00
55344,96e9bf11-c0ed-498e-98c5-a85a5384b691,NaN,Off Platform,GiveWell (Maximum Impact Fund),157994.44,USD,2021-11-09,0.758170,24b8c799-c9ce-4c10-8e99-9e8e70175ad0,157994.44
55345,e35a5e1f-47c2-4a39-b9c7-9ce6e0a11bc5,NaN,Off Platform,OFTW Top Pick: Against Malaria Foundation,250000.00,USD,2021-08-17,0.758170,ec1cfc37-8ad3-422e-9b9a-229ad9258e77,250000.00


In [43]:
df_pledges_converted

,donor_id,pledge_id,donor_chapter,chapter_type,pledge_status,date,pledge_starts_at,pledge_ended_at,contribution_amount,currency,frequency,payment_platform,amount_usd
0,NaN,fa0e5a1b-ab1d-4aba-af20-2b9d6aa23312,NaN,NaN,One-Time,2016-12-24,2016-12-24,NaT,4000.0,USD,One-Time,Fidelity DAF,4000.000
1,NaN,71bfd12a-7877-4e21-a347-7646f823149f,NaN,NaN,One-Time,2016-12-24,2016-12-24,NaT,1250.0,USD,One-Time,Fidelity DAF,1250.000
2,NaN,f8811c05-5dfd-4760-b730-8532beb2f34d,NaN,NaN,One-Time,2016-12-24,2016-12-24,NaT,1250.0,USD,One-Time,Fidelity DAF,1250.000
3,NaN,58bf5aec-b09c-4a43-ba9d-6197c1bfd7ef,NaN,NaN,One-Time,2016-12-24,2016-12-24,NaT,4000.0,USD,One-Time,Fidelity DAF,4000.000
4,NaN,32d7ecbb-e909-4881-9dd1-0875fe78fa33,NaN,NaN,One-Time,2017-07-22,2017-07-22,NaT,3000.0,USD,One-Time,Fidelity DAF,3000.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11905,NaN,bb88d99c-e831-462c-b9c4-2209a819a368,NaN,NaN,One-Time,2024-12-04,2024-12-04,NaT,60.0,GBP,One-Time,Benevity,76.272
11906,NaN,761c7af3-1b01-4852-9da0-76bd570d0fee,ZS Associates,NaN,Updated,2022-08-18,2022-08-18,2022-08-24,138.0,USD,Monthly,Benevity,138.000
11907,NaN,619cc7d3-79a0-4b3c-a027-999f902d6a84,ZS Associates,NaN,Updated,2022-10-26,2022-10-26,2023-06-28,167.0,USD,Monthly,Benevity,167.000
11908,NaN,a972e21e-9d54-477e-9385-809628f56320,ZS Associates,NaN,Updated,2023-06-27,2023-06-27,2023-10-27,350.0,USD,Monthly,Benevity,350.000
